In [27]:
# imports
import csv
import math
from typing import Dict, List, Tuple


In [ ]:

class SoilSense:
    def __init__(self, dataset_path: str):
        self.dataset = self._load_dataset(dataset_path)
        self.model = self._train_model()  # future exten

        # ideal moisture ranges
        self.moisture_ranges = {
            'Clayey': (40, 50),
            'Black': (35, 45),
            'Red': (30, 40),
            'Loamy': (20, 30),
            'Sandy': (10, 20)
        }

        # nutrient thresholds 
        self.nutrient_thresholds = {
            'nitrogen': {'low': 50, 'medium': 100},
            'phosphorus': {'low': 25, 'medium': 50},
            'potassium': {'low': 110, 'medium': 280}
        }

    def _load_dataset(self, path: str) -> List[Dict]:
        dataset = []
        try:
            with open(path, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    try:
                        dataset.append({
                            'moisture': float(row['moisture'].strip()),
                            'nitrogen': float(row['nitrogen'].strip()),
                            'phosphorus': float(row['phosphorus'].strip()),
                            'potassium': float(row['potassium'].strip()),
                            'soil_type': row['soil_type'].strip(),
                            'crop': row['crop'].strip()
                        })
                    except (ValueError, KeyError) as e:
                        print(f"Error processing row: {row}")
                        print(f"Error details: {str(e)}")
                        continue

            if not dataset:
                raise ValueError("Dataset is empty or file not found")

            print(f"successfully loaded {len(dataset)} records")
            return dataset

        except FileNotFoundError:
            raise FileNotFoundError(f"Dataset file not found: {path}")
        except Exception as e:
            raise Exception(f"Error loading dataset: {str(e)}")

    def _euclidean_distance(self, x1: Dict, x2: Dict) -> float:
        return math.sqrt(
            (x1['moisture'] - x2['moisture']) ** 2 +
            (x1['nitrogen'] - x2['nitrogen']) ** 2 +
            (x1['phosphorus'] - x2['phosphorus']) ** 2 +
            (x1['potassium'] - x2['potassium']) ** 2
        )

    def _train_model(self) -> None:
        pass

    def predict_crops(self, soil_data: Dict) -> List[Tuple[str, float]]:
        k = 5
        distances = []

        for sample in self.dataset:
            if sample['soil_type'] == soil_data['soil_type']:
                distance = self._euclidean_distance(soil_data, sample)
                distances.append((distance, sample['crop']))

        if not distances:
            return [("No suitable crop found", 0.0), ("No alternative crop", 0.0)]

        distances.sort(key=lambda x: x[0])
        neighbors = distances[:k]

        crop_counts = {}
        for _, crop in neighbors:
            crop_counts[crop] = crop_counts.get(crop, 0) + 1

        crop_confidences = [(crop, (count / k) * 100) for crop, count in crop_counts.items()]
        crop_confidences.sort(key=lambda x: x[1], reverse=True)

        if len(crop_confidences) < 2:
            crop_confidences.append(("No alternative crop", 0.0))

        return crop_confidences[:2]

    def get_fertilizer_recommendation(self, soil_data: Dict) -> str:
        n, p, k = (soil_data['nitrogen'], soil_data['phosphorus'], soil_data['potassium'])
        recommendations = []

        if n < self.nutrient_thresholds['nitrogen']['low']:
            recommendations.append(
                "High Nitrogen NPK (20-10-10):\n"
                "Addresses severe nitrogen deficiency, "
                "Promotes leafy growth and plant development.\n"
                "(উচ্চ নাইট্রোজেন এনপিকে (২০-১০-১০):\n"
                "গুরুতর নাইট্রোজেন ঘাটতি সমাধান করে, "
                "পাতা বৃদ্ধিতে সহায়তা করে)"
            )
        elif n < self.nutrient_thresholds['nitrogen']['medium']:
            recommendations.append(
                "Balanced NPK (15-15-15):\n"
                "Provides moderate nitrogen boost, "
                "Maintains overall nutrient balance.\n"
                "(সন্তুলিত এনপিকে (১৫-১৫-১৫):\n"
                "মাঝারি মাত্রার নাইট্রোজেন সরবরাহ করে, "
                "মোট পুষ্টির ভারসাম্য বজায় রাখে)"
            )

        if p < self.nutrient_thresholds['phosphorus']['low']:
            recommendations.append(
                "High Phosphorus NPK (10-20-10):\n"
                "Addresses phosphorus deficiency, "
                "Promotes root development and flowering.\n"
                "(উচ্চ ফসফরাস এনপিকে (১০-২০-১০):\n"
                "ফসফরাসের ঘাটতি দূর করে, "
                "মূল বৃদ্ধি এবং ফুল ফুটতে সহায়তা করে)"
            )

        if k < self.nutrient_thresholds['potassium']['low']:
            recommendations.append(
                "High Potassium NPK (10-10-20):\n"
                "Addresses potassium deficiency, "
                "Improves crop quality and disease resistance.\n"
                "(উচ্চ পটাশিয়াম এনপিকে (১০-১০-২০):\n"
                "পটাশিয়ামের ঘাটতি পূরণ করে, "
                "ফসলের গুণগত মান ও রোগ প্রতিরোধ ক্ষমতা বাড়ায়)"
            )

        if not recommendations:
            return ("Soil nutrients are at optimal levels. No fertilizer needed.\n"
                    "(মাটি পুষ্টি উপযুক্ত পর্যায়ে আছে। সার দেওয়ার প্রয়োজন নেই।)")

        return "\n\n".join(recommendations)

    def get_irrigation_guidance(self, soil_data: Dict) -> str:
        soil_type = soil_data['soil_type']
        current_moisture = soil_data['moisture']
        ideal_range = self.moisture_ranges[soil_type]

        if current_moisture < ideal_range[0]:
            deficit = ideal_range[0] - current_moisture
            return (
                f"Current Moisture: {current_moisture}%, "
                f"Ideal Range: {ideal_range[0]}% - {ideal_range[1]}%, "
                f"Moisture Deficit: {deficit:.1f}%\n"
                "Irrigation Recommendations:\n"
                "Frequency: 3 times per week\n"
                "Duration: 30 minutes per session\n"
                "Best Time: Early morning (before 9 AM) or evening (after 5 PM)\n"
                "Method: Drip irrigation recommended for optimal water usage\n"
                f"(বর্তমান আর্দ্রতা: {current_moisture}%, "
                f"আদর্শ পরিসর: {ideal_range[0]}% - {ideal_range[1]}%, "
                f"আর্দ্রতার ঘাটতি: {deficit:.1f}%\n"
                "সেচের সুপারিশ:\n"
                "ফ্রিকোয়েন্সি: সপ্তাহে ৩ বার\n"
                "সময়কাল: প্রতি সেশনে ৩০ মিনিট\n"
                "সর্বোত্তম সময়: সকাল আগে ৯ টা অথবা সন্ধ্যা ৫ টার পরে\n"
                "পদ্ধতি: পানির সর্বোত্তম ব্যবহারের জন্য ড্রিপ সেচ প্রয়োগ করুন)"
            )
        elif current_moisture > ideal_range[1]:
            excess = current_moisture - ideal_range[1]
            return (
                f"Current Moisture: {current_moisture}%, "
                f"Ideal Range: {ideal_range[0]}% - {ideal_range[1]}%, "
                f"Excess Moisture: {excess:.1f}%\n"
                "Recommendation: Hold irrigation until soil moisture decreases.\n"
                "Monitor drainage and avoid waterlogging.\n"
                f"(বর্তমান আর্দ্রতা: {current_moisture}%\n"
                f"আদর্শ পরিসর: {ideal_range[0]}% - {ideal_range[1]}%, "
                f"অতিরিক্ত আর্দ্রতা: {excess:.1f}%\n\n"
                "সুপারিশ: মাটির আর্দ্রতা কমা পর্যন্ত সেচ বন্ধ রাখুন। "
                "ড্রেনেজ পর্যবেক্ষণ করুন এবং পানিবন্ধ এড়ান।)"
            )
        else:
            return (
                f"Current Moisture: {current_moisture}%, "
                f"Ideal Range: {ideal_range[0]}% - {ideal_range[1]}%\n"
                "Soil moisture is within optimal range, "
                "continue regular irrigation schedule.\n"
                f"(বর্তমান আর্দ্রতা: {current_moisture}%\n"
                f"আদর্শ পরিসর: {ideal_range[0]}% - {ideal_range[1]}%\n"
                "মাটির আর্দ্রতা উপযুক্ত পর্যায়ে আছে। "
                "নিয়মিত সেচ কার্যক্রম চালিয়ে যান।)"
            )


In [51]:
# initialize the analyzer with the dataset file
analyzer = SoilSense('soilSense_ds.csv')

successfully loaded 8000 records


In [54]:
# example soil data for testing
soil_data = {
    'moisture': 0,         # moisture percentage
    'nitrogen': 35,        # nitrogen mg/kg
    'phosphorus': 11,      # phosphorus mg/kg
    'potassium': 19,       # potassium mg/kg
    'soil_type': 'Black'   # soil type
}

# predict crops with confidence scores
crops_with_confidence = analyzer.predict_crops(soil_data)


print("recommended crops:")
for i, (crop, confidence) in enumerate(crops_with_confidence):
    label = "Primary" if i == 0 else "Alternative"
    print(f"{label}: {crop}, (Confidence: {confidence:.1f}%)")

print("\nfertilizer recommendations:")
print(analyzer.get_fertilizer_recommendation(soil_data))

print("\nirrigation guidance:")
print(analyzer.get_irrigation_guidance(soil_data))


recommended crops:
Primary: Ground Nuts, (Confidence: 40.0%)
Alternative: Maize, (Confidence: 20.0%)

fertilizer recommendations:
High Nitrogen NPK (20-10-10):
Addresses severe nitrogen deficiency, Promotes leafy growth and plant development.
(উচ্চ নাইট্রোজেন এনপিকে (২০-১০-১০):
গুরুতর নাইট্রোজেন ঘাটতি সমাধান করে, পাতা বৃদ্ধিতে সহায়তা করে)

High Phosphorus NPK (10-20-10):
Addresses phosphorus deficiency, Promotes root development and flowering.
(উচ্চ ফসফরাস এনপিকে (১০-২০-১০):
ফসফরাসের ঘাটতি দূর করে, মূল বৃদ্ধি এবং ফুল ফুটতে সহায়তা করে)

High Potassium NPK (10-10-20):
Addresses potassium deficiency, Improves crop quality and disease resistance.
(উচ্চ পটাশিয়াম এনপিকে (১০-১০-২০):
পটাশিয়ামের ঘাটতি পূরণ করে, ফসলের গুণগত মান ও রোগ প্রতিরোধ ক্ষমতা বাড়ায়)

irrigation guidance:
Current Moisture: 0%, Ideal Range: 35% - 45%, Moisture Deficit: 35.0%
Irrigation Recommendations:
Frequency: 3 times per week
Duration: 30 minutes per session
Best Time: Early morning (before 9 AM) or evening (after